### Item-to-Item коллаборативная фильтрация

Рекомендации фильма на основе рейтинга фильмов

In [3]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('data/ml-25m/movies.csv')
ratings = pd.read_csv('data/ml-25m/ratings.csv')

In [3]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [ ]:
#set_index on (присоединение индекса movieId по индексу movieId)
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)

In [ ]:
movies_with_ratings.head(5)

In [7]:
#удалим NaN
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings.userId

0                2.0
1                3.0
2                4.0
3                5.0
4                8.0
              ...   
25003466    119571.0
25003467    115835.0
25003468      6964.0
25003469    119571.0
25003470    119571.0
Name: userId, Length: 25000095, dtype: float64

In [9]:
#уникальное число пользователей
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

162541

In [34]:
movie_vector = {}
#группировка строк по заголовкам
for movie, group in movies_with_ratings.groupby('title'):
    #сперва вектор заполнен нулями, т.е если пользователь не смотрел фильм
    #то будет 0
    movie_vector[movie]=np.zeros(num_users)
    #заполняем вектор
    for i in range(len(group.userId.values)):
        user = group.userId.values[i]
        rating = group.rating.values[i]
        movie_vector[movie][int(user-1)]=rating

In [12]:
#импортируем расстояние
from scipy.spatial.distance import cosine, euclidean, hamming, correlation

In [64]:
type(movie_vector)

dict

In [66]:
#вывод вектора (матрица с фильмами и пользователями (0 если пользователь не смотрел фильм))
#вывод items() возвращает пары(ключ, значение)
list(movie_vector.items())[:5]

[('"BLOW THE NIGHT!" Let\'s Spend the Night Together (1983)',
  array([0., 0., 0., ..., 0., 0., 0.])),
 ('"Great Performances" Cats (1998)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#1 Cheerleader Camp (2010)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#Captured (2017)', array([0., 0., 0., ..., 0., 0., 0.])),
 ('#Female Pleasure (2018)', array([0., 0., 0., ..., 0., 0., 0.]))]

In [70]:
favorite_film = 'Toy Story (1995)'

titles = []
distances = []
#смотрим как коррелируют между собой фильмы (ищим похожий фильм)
#опираясь на рейтинг
for key in movie_vector.keys():
    if key == favorite_film:
        continue
    
    titles.append(key)
    distances.append(correlation(movie_vector[favorite_film], movie_vector[key]))
      
best_indexes = np.argsort(distances)[:10]
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for movie in best_movies:
    print(movie)

('Toy Story 2 (1999)', 0.551006507194897)
('Willy Wonka & the Chocolate Factory (1971)', 0.6511756810314209)
('Back to the Future (1985)', 0.6562068418410386)
('Lion King, The (1994)', 0.656776432864756)
("Bug's Life, A (1998)", 0.6599777634641218)
('Aladdin (1992)', 0.6614160096207382)
('Independence Day (a.k.a. ID4) (1996)', 0.6617913359976116)
('Monsters, Inc. (2001)', 0.664391428963012)
('Mission: Impossible (1996)', 0.6797301234472074)
('Star Wars: Episode IV - A New Hope (1977)', 0.6807438359095375)


Чем меньше корреляция, тем больше соответсвие

Пользователям которым понравился фильм Toy Story (1995) нравятся ещё фильмы перечисленные выше

### User-Based коллаборативная фильтрация

In [57]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

In [58]:
movies = pd.read_csv('data/ml-25m/movies.csv', nrows=10000)
ratings = pd.read_csv('data/ml-25m/ratings.csv', nrows=10000)
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [63]:
#фильмы которые смотрел пользователь 2
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()[:20]

array(['Toy Story (1995)', "Mr. Holland's Opus (1995)",
       'Braveheart (1995)', 'Apollo 13 (1995)', 'Rob Roy (1995)',
       'French Kiss (1995)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Little Women (1994)', 'Legends of the Fall (1994)',
       'Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Clear and Present Danger (1994)', 'Forrest Gump (1994)',
       'Lion King, The (1994)', 'True Lies (1994)',
       'Fugitive, The (1993)', 'Jurassic Park (1993)',
       'Much Ado About Nothing (1993)', 'Rudy (1993)',
       "Schindler's List (1993)"], dtype=object)

In [64]:
dataset = pd.DataFrame({
    'user_id': movies_with_ratings.userId,
    'item_id': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [65]:
dataset.head()

,user_id,item_id,rating
0,2.0,Toy Story (1995),3.5
1,3.0,Toy Story (1995),4.0
2,4.0,Toy Story (1995),3.0
3,5.0,Toy Story (1995),4.0
4,8.0,Toy Story (1995),4.0


In [66]:
ratings.rating.min()

0.5

In [67]:
ratings.rating.max()

5.0

In [68]:
reader = Reader(rating_scale=(0.5, 5.0))#парсер (указали минимальную и максимальную оценку)
data = Dataset.load_from_df(dataset, reader)#для работы surprise
data #разряженная матрица (матрица пар пользователь - объект)
#если пользователь не взаимодействовал с объектом - пересечние = 0
#0 не хранит (не воспримает, для экономия оперативной памяти)

In [69]:
from surprise.model_selection import train_test_split

In [70]:
X_train, X_test = train_test_split(data, test_size=.15)

In [72]:
X_test[:10]

[(19.0, 'Fifth Element, The (1997)', 3.5),
 (54.0, 'Sabrina (1995)', 3.0),
 (38.0, 'Star Wars: Episode IV - A New Hope (1977)', 4.5),
 (41.0, 'Jaws (1975)', 3.0),
 (36.0, 'Secret of Roan Inish, The (1994)', 4.0),
 (67.0, 'Super Mario Bros. (1993)', 1.0),
 (59.0, 'Ladyhawke (1985)', 3.0),
 (59.0, 'Jurassic Park (1993)', 4.0),
 (12.0, 'D2: The Mighty Ducks (1994)', 1.0),
 (13.0, 'Austin Powers: The Spy Who Shagged Me (1999)', 3.5)]

In [73]:
#pearson_baseline - корреляция
#user_based = True --> User Based рекомендательная система
#user_based = False --> item to item
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
knn.fit(X_train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [74]:
pred = knn.test(X_test)#test - оценивает значения

In [75]:
accuracy.rmse(pred, verbose=True)

RMSE: 1.0125


1.0124580069317604

Высокая ошибка, ошибаемся на 1 балл (завышаем или занижаем оценку)

In [76]:
#предсказываем, какой рейтинг пользователь 2 поставит фильму Pulp Fiction (1994)
#uid - user_id
#iid - item_id
result = knn.predict(uid=2, iid='Pulp Fiction (1994)')

In [77]:
#r_ui=None - Человек не смотрел фильм (нет оценки)
result

Prediction(uid=2, iid='Pulp Fiction (1994)', r_ui=None, est=4.239442716893183, details={'actual_k': 10, 'was_impossible': False})

In [78]:
print('Оценка пользователя 2 по мнению алгоритма'+' = '+ str(result.est))

Оценка пользователя 2 по мнению алгоритма = 4.239442716893183


### Item-Based коллаборативная фильтрация

Лучше работает, когда пользователей больше, чем объектов

In [79]:
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
knn.fit(X_train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [80]:
pred = knn.test(X_test)

In [81]:
accuracy.rmse(pred, verbose=True)

RMSE: 0.9938


0.99379913768205

Качество модели немного лучше, чем при использовании User-Based

In [82]:
result = knn.predict(uid=2, iid='Pulp Fiction (1994)')

In [83]:
result

Prediction(uid=2, iid='Pulp Fiction (1994)', r_ui=None, est=4.354341184598326, details={'actual_k': 50, 'was_impossible': False})

In [84]:
print('Оценка пользователя 2 по мнению алгоритма'+' = '+ str(result.est))

Оценка пользователя 2 по мнению алгоритма = 4.354341184598326


In [90]:
dataset.user_id.unique().shape[0], dataset.item_id.unique().shape[0]

(75, 2431)

Пользователей меньше, чем объектов, следовательно больше подходит User-Based. Качество лучше из-за маленького датасета

### Тоже самое на большом dataset'е

In [ ]:
movies = pd.read_csv('data/ml-25m/movies.csv')
ratings = pd.read_csv('data/ml-25m/ratings.csv')
movies_with_ratings = movies.join(ratings.set_index('movieId'),on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
dataset = pd.DataFrame({
    'user_id': movies_with_ratings.userId,
    'item_id': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader

In [ ]:
X_train, X_test = train_test_split(data, test_size=.15)

#### User-Based

In [ ]:
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
pred = knn.fit(X_train).test(X_test)
accuracy.rmse(pred, verbose=True)

#### Item-Based

In [ ]:
knn = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
pred = knn.fit(X_train).test(X_test)
accuracy.rmse(pred, verbose=True)